# Read an LTO File

[Struct Library](https://docs.python.org/3/library/struct.html)


In [1]:
from struct import *
import numpy as np

In [2]:
path = 'Test-flat.LTO'

## LTO File Header Structure
...is divided into several sections as follows:

In [3]:
# empty dictionary to hold the header structure
LTO_File_Hdr = {}

In [4]:
# magic number at top of file
LTO_File_Hdr["Magic"] =  [
    ('magic',   int, 'i'),
    ('version', int, 'i')
]

In [5]:
# observatory
LTO_File_Hdr['Observatory'] = [
    ('obs_name', str, '32s'),
    ('obs_id',   int, 'i'),
    ('ref_type', str, '32s'),
    ('clock_type',str,'32s'),
    ('rx_type',  str, '32s')
]

In [6]:
# obs loc
#f_obsloc_tup = namedtuple('F_Obsloc_tup',['lat', 'lon', 'alt'])
#f_obsloc_fmt = '<3f'
LTO_File_Hdr['ObsLocation'] = [
    ('lat', float, 'f'),
    ('lon', float, 'f'),
    ('alt', float, 'f')
]

In [7]:
# beam pos
LTO_File_Hdr['BeamPosition'] = [
    ('az',    float, 'f'),
    ('el',    float, 'f'),
    ('dec',   float, 'f'),
    ('ra',    float, 'f'),
    ('user1', str,   '64s')
]

In [8]:
#observing time

LTO_File_Hdr['ObsTime'] = [
    ('year',   int, 'l'),
    ('daynum', int, 'l'),
    ('month',  int, 'l'),
    ('day',    int, 'l'),
    ('hour',   int, 'l'),
    ('minute', int, 'l'),
    ('second', float, 'f'),
    ('epoch',  int,  'q'), #q = 8 byte int
    ('julian', float, 'd'), #d = double maps to python float
    ('GST',    float, 'd'),
    ('LST',    float, 'd'),
    ('readme', str, '128s'),
    ('user2',  str, '64s')
]

In [9]:
#Spectrum:
LTO_File_Hdr['Spectrum'] =[
    ('samplerate',  float, 'f'),
    ('wordformat',  int,   'l'),
    ('numchannels', int,   'l'),
    ('lenfft',      int,   'l'),
    ('numspec',     int,   'l'),
    ('numave',      int,   'l'),
    ('period',      float, 'f'),
    ('dfs',         float, 'f'),
    ('cf',          float, 'd'), # 8 byte float
    ('nenbw',       float, 'f'),
    ('enbw',        float, 'f'),
    ('ws1',         float, 'f'),
    ('ws2',         float, 'f'),
    ('dt',          float, 'f'),
    ('user3',       str,   '64s')
]

In [10]:
#Radio Calibrations
LTO_File_Hdr['RadioCalibrations'] = [
    ('g0feed',      float, 'f'),
    ('g0rx',        float, 'f'),
    ('g03',         float, 'f'),
    ('feedtemp',    float, 'f'),
    ('rxtemp',      float, 'f'),
    ('temp3',       float, 'f'),
    ('noisefigure', float, 'f'),
    ('dgdtf',       float, 'f'),
    ('dgdtr',       float, 'f'),
    ('dgdt3',       float, 'f'),
    ('tsys',        float, 'f'),
    ('aeff',        float, 'f'),
    ('pcal',        float, 'f'),
    ('gsys',        float, 'd'),
    ('user4',       str,   '64s'),
]

In [11]:
#Program Control
LTO_File_Hdr['ProgramControl'] = [
    ('fscale',  float,'f'),
    ('flatwfm', bool, 'L'), # read these as 4-byte unsigned longs
    ('flatirq', bool, 'L'),
    ('bogie',   bool, 'L'),
    ('smooth',  bool, 'L'),
    ('spikes',  bool, 'L'),
    ('rmlo',    bool, 'L'),
    ('whole',   bool, 'L'),
    ('window',  bool, 'L'),
    ('winnum',  int,  'l'),
    ('overlap', float,'f'),
    ('user5',   str,  '64s')
]

In [12]:
LTO_File_Hdr['SpectralCharacteristics'] = [
    ('flow', float, 'f'),
    ('fhigh', float, 'f'),
    ('ndlow', int, 'l'),
    ('ndhigh', int, 'l'),
    ('avespecpwr', float, 'f'),
    ('varspecpwr', float, 'f'),
    ('numspecpwr', int, 'l'),
    ('peakpwr', float, 'f'),
    ('peakpwrfreq', float, 'd'),
    ('totalHIpwr', float, 'f'),
    ('numHIpwr', int, 'l'),
    ('avecrpwr', float, 'f'),
    ('varcrpwr', float, 'f'),
    ('numcrpwr', int, 'l'),
    ('avetsky', float, 'f'),
    ('vartsky', float, 'f'),
    ('peaktsky', float, 'f'),
    ('peaktskyfreq', float, 'f'),
    ('avefluxden', float, 'f'),
    ('varfluxden', float, 'f'),
    ('peakfluxden', float, 'f'),
    ('peakfluxfreq', float, 'f'),
    ('badspec', bool, 'L'),
    ('processing', str, '64s'),
    ('user', str, '64s')
    
]

## Data Arrays
The actual spectral data follows the header in the data file. Each array is a contiguous block of LenFFT elements.  Element types (and therefore their size in the file) vary

In [13]:
#these guys are all arrays of len = lenfft
LTO_File_Data = {}
LTO_File_Data['data'] = [
    ('dopfreq',   float, 'f'),
    ('rawavepwr', float, 'f'),
    ('rawvarpwr', float, 'f'),
    ('calavepwr', float, 'f'),
    ('calvarpwr', float, 'f'),
    ('tsky',      float, 'f'),
    ('tskyvar',   float, 'f'),
    ('fluxden',   float, 'f'),
    ('fluxdenvar',float, 'f'),
    ('badline',   bool,  'L'), #booleans read as unsigned longs
    ('HIline',    bool,  'L')
]

## Read the Header
Data arrays later. This bombs out on the Spectral Charcteristics section of the header, doesn't make it down to the assertion.

In [14]:
lto_hdr = {}
with open(path, 'rb') as fin:
    
    # read the file header
    for sect in LTO_File_Hdr: # loop thru the sections and read the members of each
        sect_dict = {}
        for f in LTO_File_Hdr[sect]:
            fmt = '<'+ f[2]
            n = calcsize(fmt)
            buf = fin.read(n)
            v = unpack(fmt, buf)[0]
            if f[1] == str:
                v = v.decode('utf8').strip()
            elif f[1] == bool:
                v = bool(v != 0)
            sect_dict[f[0]] = v
            
        lto_hdr[sect] = sect_dict
        
    assert(False) #bomb out if we get this far (deal with data arrays later)
    
    # read the data arrays
    nelems = lto_hdr['Spectrum']['lenfft'] # each array is this long
    data = {}
    for d in data_arrays['vals']:
        fmt = '<' + str(nelems) + d[2]
        n = calcsize(fmt)
        buf = fin.read(n)
        if d[1] == bool:
            data[d[0]] = (np.array(unpack(fmt, buf)) != 0)
        else:
            data[d[0]] = np.array(unpack(fmt, buf))
    lto_file = {'Header': lto_hdr, 'Data': data}

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 16: invalid start byte

In [16]:
# section and field that it bombed on:
sect, f[0]

('SpectralCharacteristics', 'user')

## Verification

In [17]:
def printsection(hdr, sect):
    print ('\nHeader Section: '+ sect)
    for f in LTO_File_Hdr[sect]:
        print('\t'+f[0]+': '+str(hdr[sect][f[0]]))
        

In [18]:
for sect in LTO_File_Hdr:
    printsection(lto_hdr,sect)


Header Section: Magic
	magic: 1330924544
	version: 1

Header Section: Observatory
	obs_name: Little Thompson Observatory
	obs_id: 1
	ref_type: OCXO
	clock_type: Windows NTP
	rx_type: Airspy

Header Section: ObsLocation
	lat: 40.29953384399414
	lon: -105.08438873291016
	alt: 1534.0

Header Section: BeamPosition
	az: 180.0
	el: 21.0
	dec: -28.70046615600586
	ra: 15.03388500213623
	user1: SDR# Gains=8,8,11

Header Section: ObsTime
	year: 2018
	daynum: 737328
	month: 11
	day: 25
	hour: 17
	minute: 43
	second: 38.0
	epoch: 0
	julian: 2458448.2386342683
	GST: 22.039510572969448
	LST: 15.033884657442105
	readme: 
	user2: 

Header Section: Spectrum
	samplerate: 10000000.0
	wordformat: 0
	numchannels: 2
	lenfft: 16384
	numspec: 1
	numave: 15076
	period: 0.001638400019146502
	dfs: 610.3515625
	cf: 0.0
	nenbw: 1.0
	enbw: 610.3515625
	ws1: 1.0
	ws2: 16384.0
	dt: 24.700517654418945
	user3: 

Header Section: RadioCalibrations
	g0feed: 18.0
	g0rx: 158.0
	g03: 0.0
	feedtemp: 290.0
	rxtemp: 290.0
	tem

KeyError: 'SpectralCharacteristics'

## Spectral Characteristics So Far
Note that the first two struct members, `flow` and `fhigh` match their values in the text .fft file. However the next value `ndlow` doens't match.

In [19]:
for f in LTO_File_Hdr['SpectralCharacteristics']:
    print('\t'+ f[0]+': '+str(sect_dict[f[0]]))

	flow: -3000000.0
	fhigh: 3000000.0
	ndlow: 748209369
	ndhigh: 378736672
	avespecpwr: 4.7120783837339975e-12
	varspecpwr: -0.0
	numspecpwr: 1092096188
	peakpwr: 0.0
	peakpwrfreq: 0.0
	totalHIpwr: 0.0
	numHIpwr: 0
	avecrpwr: 2.095391438746426e-19
	varcrpwr: 1.798035001182215e-19
	numcrpwr: 538976288
	avetsky: 1.3563156426940112e-19
	vartsky: 1.3563156426940112e-19
	peaktsky: 1.3563156426940112e-19
	peaktskyfreq: 1.3563156426940112e-19
	avefluxden: 1.3563156426940112e-19
	varfluxden: 1.3563156426940112e-19
	peakfluxden: 1.3563156426940112e-19
	peakfluxfreq: 1.3563156426940112e-19
	badspec: True
	processing: 


KeyError: 'user'

## Diagnostic Tools

In [20]:
# returns nwords of the file starting at offset (4-byte words)
def getfilebytes(path, offset, nwords):
    #4 byte words
    with open(path,'rb') as fin:
        fin.seek(offset)
        buf = fin.read(nwords*4)
    return buf

    #fmt = '<'+str(nwords)+'L' #read in as unsigned ints
    #vals = unpack(fmt, buf)
        
    #hexlist = ['0x%0*X' % (8,v) for v in vals]
    #hexlist = ['0x%0*X' % (8,int(buf[v:v+4],32)) for v in range(0, len(buf),4)]
    #return hexlist,buf
        

In [21]:
def hex2float(s):
    #skip past the '0x'
    bins = bytes([int(s[x:x+2], 16) for x in range(2, len(s), 2)])
    return struct.unpack('<f', bins)[0]

In [22]:
def bytes2hexwords(b):
    #assume 4-byte words
    l = len(b)
    return np.array(['0x'+''.join(['%0*X' % (2,b[j]) for j in range(i,i+4)]) for i in range(0,l,4)])

In [23]:
def bytes2ascii(b):
    return '-'.join([chr(i) for i in b])

In [24]:
def getval(b,fmt):
    return( unpack('<'+fmt, b))

In [25]:
#test by reading first 8 words of file, which will be the magic and into the observatory section
b = getfilebytes(path,0,8)

In [26]:
bytes2hexwords(b)

array(['0x004C544F', '0x01000000', '0x4C697474', '0x6C652054',
       '0x686F6D70', '0x736F6E20', '0x4F627365', '0x72766174'],
      dtype='<U10')

In [27]:
bytes2ascii(b)

'\x00-L-T-O-\x01-\x00-\x00-\x00-L-i-t-t-l-e- -T-h-o-m-p-s-o-n- -O-b-s-e-r-v-a-t'

In [28]:
getval(b[0:8],'ii')

(1330924544, 1)

In [29]:
#magic number
hex(getval(b[0:4],'i')[0])

'0x4f544c00'

In [30]:
# the observatory section of the header starts at byte offset 8
bytes2ascii(b[8:])

'L-i-t-t-l-e- -T-h-o-m-p-s-o-n- -O-b-s-e-r-v-a-t'

## Compute Offsets for Each Header Member

In [31]:
import pandas as pd

In [32]:
hdr_df = pd.DataFrame([(s,v[0], v[1],v[2])  for s in LTO_File_Hdr for v in LTO_File_Hdr[s]],
                     columns=['Section', 'Field', 'PythonType','Format'])
hdr_df['FileSize'] = [calcsize('<'+f.Format) for f in hdr_df.itertuples()]
hdr_df['Offest'] = [0]+hdr_df.FileSize.cumsum()[0:len(hdr_df)-1].tolist()

In [33]:
hdr_df.head()

,Section,Field,PythonType,Format,FileSize,Offest
0,Magic,magic,<class 'int'>,i,4,0
1,Magic,version,<class 'int'>,i,4,4
2,Observatory,obs_name,<class 'str'>,32s,32,8
3,Observatory,obs_id,<class 'int'>,i,4,40
4,Observatory,ref_type,<class 'str'>,32s,32,44


## Read the Spectral Characteristics Section

In [34]:
hdr_df.query('Section == \'SpectralCharacteristics\'')

,Section,Field,PythonType,Format,FileSize,Offest
70,SpectralCharacteristics,flow,<class 'float'>,f,4,840
71,SpectralCharacteristics,fhigh,<class 'float'>,f,4,844
72,SpectralCharacteristics,ndlow,<class 'int'>,l,4,848
73,SpectralCharacteristics,ndhigh,<class 'int'>,l,4,852
74,SpectralCharacteristics,avespecpwr,<class 'float'>,f,4,856
75,SpectralCharacteristics,varspecpwr,<class 'float'>,f,4,860
76,SpectralCharacteristics,numspecpwr,<class 'int'>,l,4,864
77,SpectralCharacteristics,peakpwr,<class 'float'>,f,4,868
78,SpectralCharacteristics,peakpwrfreq,<class 'float'>,d,8,872
79,SpectralCharacteristics,totalHIpwr,<class 'float'>,f,4,880


In [35]:
#look at the first 20 words of the SpectralCharacteristics section which starts at offset 840
b = getfilebytes(path, 840, 20)

In [36]:
bytes2hexwords(b)

array(['0x001B37CA', '0x001B374A', '0xD9C4982C', '0x20109316',
       '0xA1CAA52C', '0x00000080', '0xBC101841', '0x00000000',
       '0x00000000', '0x00000000', '0x00000000', '0x00000000',
       '0x52617720', '0x46465420', '0x20202020', '0x20202020',
       '0x20202020', '0x20202020', '0x20202020', '0x20202020'],
      dtype='<U10')

First two words are `Flow` and `FHigh` respectively

In [37]:
getval(b[0:8],'ff') # two floats

(-3000000.0, 3000000.0)

These match the expected values.

Next two words are `NDlow` and `NDhigh` respectively

In [38]:
getval(b[8:16],'ll') # two longs (4-byte ints)

(748209369, 378736672)

These are incorrect. The values should be (-4915, 4915) respectively

Note some ASCII data starting at 48 bytes past the start of the Spectral Characteristics section. This is at file offset 888, which according to the Fortran header file, should be the start of the field `avecwpwr`.

Note, the hyphens in the output below separate the individual ASCII characters to improve readability.

In [39]:
bytes2ascii(b[44:])

'\x00-\x00-\x00-\x00-R-a-w- -F-F-T- - - - - - - - - - - - - - - - - - - - - - - - - '